<a href="https://colab.research.google.com/github/CalebEng/Ai-Project/blob/master/Ai_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#imports
import torch
from torch import nn

!pip install --upgrade tiktoken
import tiktoken
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Ai work')
#torch.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



------------------- Prep and loading data sets -------------------

--ONLY RUN THE NEXT IF CSV FILE CONTAINING CONVOS HAS A INDEX COLUMN--

In [6]:
import pandas as pd
df = pd.read_csv('Conversation.csv')
# If you know the name of the column skip this
first_column = df.columns[0]
# Delete first
df = df.drop([first_column], axis=1)
df.to_csv('Conversation.csv', index=False)


In [48]:
df = pd.read_csv('Conversation.csv')

In [49]:
def format(x):
  return enc.encode(x)
df['question'] = df['question'].map(format)
df['answer'] = df['answer'].map(format)


In [50]:
df.to_csv('Conversation_enc.csv',index=False)

--EXIT--

In [ ]:

data_q = torch.tensor(questions)
data_a = torch.tensor(answers)


In [58]:
enc = tiktoken.encoding_for_model('gpt-3.5-turbo')

df2 = pd.read_csv('Conversation.csv')

questions = df2['question'].tolist()
answers = df2['answer'].tolist()

temp = map(format, questions)
temp2 = map(format, answers)

questions = list(temp)
answers = list(temp2)


In [60]:
questions


[[6151, 11, 1268, 527, 499, 3815, 30],
 [72, 2846, 7060, 13, 1268, 922, 6261, 30],
 [72, 2846, 5128, 1695, 13, 9523, 369, 10371, 13],
 [2201, 3575, 13, 779, 1268, 617, 499, 1027, 30],
 [72, 3077, 1027, 2294, 13, 1148, 922, 499, 30],
 [72, 3077, 1027, 1695, 13, 602, 2846, 304, 2978, 1314, 1457, 13],
 [12840, 2978, 656, 499, 733, 311, 30],
 [72, 733, 311, 281, 641, 13],
 [3055, 499, 1093, 433, 1070, 30],
 [275, 596, 17339, 13, 433, 596, 264, 2216, 2466, 15679, 13],
 [19045, 15369, 449, 2978, 13],
 [5269, 596, 433, 2133, 30],
 [72, 2846, 3815, 1664, 13, 1268, 922, 499, 30],
 [37593, 2731, 11, 9523, 13],
 [708, 1268, 617, 499, 1027, 31445, 30],
 [72, 3077, 3604, 1027, 5128, 1695, 13, 499, 30],
 [72, 2846, 3604, 304, 2978, 1314, 1457, 13],
 [8370, 2978, 656, 499, 9604, 30],
 [72, 2846, 24096, 281, 641, 1314, 1457, 13],
 [548, 499, 22128, 433, 1070, 30],
 [275, 596, 539, 3958, 13, 1070, 527, 264, 2763, 315, 1274, 1070, 13],
 [19045, 15369, 449, 430, 13],
 [5269, 527, 499, 3815, 3432, 30],
 [

In [59]:
data_q= torch.tensor(questions)
data_a= torch.tensor(answers)


ValueError: ignored

In [ ]:
data_q